In [2]:
pip install growwapi pyotp

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datetime import datetime, timedelta
import time
import pyotp
from growwapi import GrowwAPI
import pandas as pd
import threading
from ta.trend import SMAIndicator
 
# API credentials for Groww trading platform
credentials = {
    'api_key': "Your API Key here",
    'totp_secret': "Your API secret here"
}
 
# Strategy configuration - Moving Average Crossover parameters
config = {
    "exchange": "NSE",                    # Exchange (NSE / BSE)
    "trading_symbol": "RELIANCE",         # Stock symbol to trade
    "interval": 1,                        # Candle interval in minutes
    "sma_short": 20,                      # Short-term Simple Moving Average period
    "sma_long": 50,                       # Long-term Simple Moving Average period
    "quantity": 1,                        # Number of shares to trade
    "entry_time": "10:00:00",             # Market entry time (after market opens)
    "exit_time": "15:00:00"               # Market exit time (before market closes)
}
 
class Candles:
    """
    Manages real-time and historical candle data for technical analysis.
    Handles data fetching, storage, and automatic updates in a thread-safe manner.
    """
    def __init__(self, groww, instrument, interval = 1):
        self.groww = groww                    # Groww API instance
        self.instrument = instrument          # Trading instrument details
        self.interval = interval              # Candle interval in minutes
        self.last_updated_at = None           # Timestamp of last update
        self.candles = None                   # DataFrame containing candle data
        self.data_lock = threading.Lock()     # Thread lock for data safety
 
    def __get_historic_candles(self, duration):
        """Fetch historical candle data for specified duration in days"""
        range_to = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
        range_from = range_to - timedelta(duration)
        return self.__get_candles_from_groww(range_from, range_to)
 
    def __get_candles_from_groww(self, start, end = None):
        """Fetch candle data from Groww API and convert to DataFrame"""
        response = self.groww.get_historical_candle_data(
            trading_symbol=self.instrument['trading_symbol'],
            exchange=self.instrument['exchange'],
            segment=self.instrument['segment'],
            start_time=int(start.timestamp() * 1000),  # Convert to milliseconds
            end_time=int(end.timestamp() * 1000) if end else int(datetime.today().timestamp() * 1000),
            interval_in_minutes=self.interval
        )
        # Create a DataFrame with OHLCV data
        columns = ['timestamp','open','high','low','close','volume']
        candles_df = pd.DataFrame(response['candles'], columns=columns)
        candles_df.set_index("timestamp", inplace=True)  # Set timestamp as index
        return candles_df
 
    def __sleep_until_next_minute(self):
        """Sleep until the start of the next minute for precise timing"""
        now = datetime.now()
        next_minute = (now + timedelta(minutes=1)).replace(second=0, microsecond=0)
        time.sleep((next_minute - now).total_seconds())
 
    def __update_candles(self):
        """Update candle data with latest market data"""
        start_time = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
        self.update_time = datetime.now()
        new_candles = self.__get_candles_from_groww(start_time)
        combined_df = None
 
        with self.data_lock:
            if self.candles is None or len(self.candles) == 0:
                # First time loading data
                self.candles = new_candles
            elif new_candles is None or len(new_candles) == 0:
                # No new data available
                return
            else:
                # Combine existing and new data, remove duplicates
                self.candles = pd.concat([self.candles, new_candles])
                self.candles = self.candles[~self.candles.index.duplicated(keep='last')]
                self.candles.sort_index(inplace=True)
 
    def __auto_update_candles(self, refresh_interval):
        """Background thread function to continuously update candle data"""
        while True:
            self.__update_candles()
            time.sleep(refresh_interval)
 
    def load(self, duration = 0):
        """Load initial candle data - historical data if duration > 0"""
        with self.data_lock:
            self.candles = self.__get_historic_candles(duration) if duration > 0 else None
        
        self.__update_candles()
        
    def start_auto_update(self, refresh_interval = 5):
        """Start background thread for automatic data updates"""
        t = threading.Thread(target=self.__auto_update_candles, daemon=True, args=(refresh_interval,))
        t.start()
 
    def get_total_candles_count(self):
        """Get total number of candles in memory"""
        with self.data_lock:
            return len(self.candles)
    
    def get(self, count = None, completed = False):
        """
        Get candle data with optional filtering
        count: Number of recent candles to return
        completed: If True, exclude the current incomplete candle
        """
        comp_time = int(self.update_time.timestamp() / 60) * 60  # Round to minute
        filtered_candles = None
        with self.data_lock:
            if completed:
                last_candle_ts = self.candles.index[-1]
                
                if last_candle_ts >= comp_time:
                    # Exclude current incomplete candle
                    filtered_candles = self.candles[:-1]
            
            filtered_candles = self.candles.copy() if filtered_candles is None else filtered_candles
            return filtered_candles.tail(count) if count else filtered_candles
 
class Instrument:
    """
    Manages instrument data and provides methods to find specific instruments
    (stocks, futures, options) from the instruments master data.
    """
    def __init__(self, groww):
        self.instruments = groww.get_all_instruments()  # Load all available instruments
 
    def __get_stock_index(self, exchange, trading_symbol):
        """Find stock or index instrument by exchange and symbol"""
        instrument = self.instruments[
                            (self.instruments['exchange'] == exchange) &
                            (self.instruments['trading_symbol'] == trading_symbol)]
        if len(instrument) <= 0: 
            raise ValueError("Instrument not found")
        else:
            return instrument.iloc[0].to_dict()
 
    def __get_future(self, exchange, trading_symbol, expiry_date):
        """Find futures contract by exchange, symbol, and expiry"""
        instrument = self.instruments[
                            (self.instruments['exchange'] == exchange) &
                            (self.instruments['underlying_symbol'] == trading_symbol) &
                            (self.instruments['instrument_type'] == 'FUT') &
                            (self.instruments['expiry_date'] == expiry_date)]
        if len(instrument) <= 0: 
            raise ValueError("Instrument not found")
        else:
            return instrument.iloc[0].to_dict()
 
    def __get_option(self, exchange, trading_symbol, expiry_date, strike_price, option_type):
        """Find option contract by exchange, symbol, expiry, strike, and type (CE/PE)"""
        instrument = self.instruments[
                            (self.instruments['exchange'] == exchange) &
                            (self.instruments['underlying_symbol'] == trading_symbol) &
                            (self.instruments['instrument_type'] == option_type) &
                            (self.instruments['expiry_date'] == expiry_date) &
                            (self.instruments['strike_price'] == strike_price)]
        if len(instrument) <= 0: 
            raise ValueError("Instrument not found")
        else:
            return instrument.iloc[0].to_dict()
 
    def get_instrument(self, exchange, trading_symbol, expiry_date = None, strike_price = None, option_type = None):
        """
        Unified method to get any type of instrument
        - If no expiry: returns stock/index
        - If expiry but no strike: returns futures
        - If expiry and strike: returns options
        """
        if expiry_date is None:
            return self.__get_stock_index(exchange, trading_symbol)
        elif strike_price is None:
            return self.__get_future(exchange, trading_symbol, expiry_date)
        else:
            return self.__get_option(exchange, trading_symbol, expiry_date, strike_price, option_type)
 
def init_groww(api_key, totp_secret):
    """
    Initialize Groww API with TOTP authentication
    Generates TOTP code and gets access token for API calls
    """
    totp_gen = pyotp.TOTP(totp_secret)
    totp = totp_gen.now()
    access_token = GrowwAPI.get_access_token(api_key, totp)
    return GrowwAPI(access_token)
 
def await_entry(entry_time):
    """Wait until the specified entry time before starting trading"""
    start_time = datetime.strptime(entry_time, "%H:%M:%S").time()
    while datetime.now().time() < start_time:
        time.sleep(1)
        print(f"Waiting for entry time {start_time}")
 
def is_exit_time(exit_time):
    """Check if current time has reached or passed the exit time"""
    end_time = datetime.strptime(exit_time, "%H:%M:%S").time()
    return datetime.now().time() >= end_time
 
def get_sma(candles_obj, sma_short, sma_long):
    """
    Calculate Simple Moving Averages and detect crossover signals
    Returns: (direction, crossover_detected)
    - direction: "BULLISH" or "BEARISH" based on current SMA positions
    - crossover_detected: True if a crossover occurred in the last period
    """
    # Get enough candles for both SMAs plus one extra for comparison
    candles = candles_obj.get(count=sma_long + 1, completed=True)
    
    # Calculate both moving averages
    sma_short_series = SMAIndicator(pd.Series(candles['close']), sma_short).sma_indicator()
    sma_long_series = SMAIndicator(pd.Series(candles['close']), sma_long).sma_indicator()
 
    # Current and previous values for both SMAs
    sma_short_c = sma_short_series.iloc[-1]  # Current short SMA
    sma_long_c = sma_long_series.iloc[-1]    # Current long SMA
    sma_short_p = sma_short_series.iloc[-2]  # Previous short SMA
    sma_long_p = sma_long_series.iloc[-2]    # Previous long SMA
 
    direction = None
    cross_over = False
    direction_p = None
 
    # Check if we have valid current values
    if pd.isna(sma_short_c) or pd.isna(sma_long_c):
        return direction, cross_over
 
    # Determine current market direction
    if sma_short_c >= sma_long_c:
        direction = "BULLISH"
    elif sma_short_c < sma_long_c:
        direction = "BEARISH"
    
    # Check if we have valid previous values for crossover detection
    if pd.isna(sma_short_p) or pd.isna(sma_long_p):
        return direction, cross_over
 
    if sma_short_p >= sma_long_p:
        direction_p = "BULLISH"
    elif sma_short_p < sma_long_p:
        direction_p = "BEARISH"
 
    if direction_p and direction and direction_p != direction:
        cross_over = True
    
    return direction, cross_over
 
def buy_at_market(groww, instrument, quantity, intraday = True):
    """
    Place a market buy order
    intraday: If True, uses MIS (Margin Intraday Square-off) product
    """
    try:
        orderid = groww.place_order(
            trading_symbol=instrument['trading_symbol'],
            quantity=quantity,
            validity=groww.VALIDITY_DAY,
            exchange=instrument['exchange'],
            segment=instrument['segment'],
            product=groww.PRODUCT_MIS if intraday else groww.PRODUCT_CNC if instrument['segment'] == groww.SEGMENT_CASH else groww.PRODUCT_NRML,
            order_type=groww.ORDER_TYPE_MARKET,
            transaction_type=groww.TRANSACTION_TYPE_BUY,
        )
        print(f"Buy order executed: {orderid}")
        return orderid
    except Exception as e:
        print(e)
        return None
 
def sell_at_market(groww, instrument, quantity, intraday = True):
    """
    Place a market sell order
    intraday: If True, uses MIS (Margin Intraday Square-off) product
    """
    try:
        orderid = groww.place_order(
            trading_symbol=instrument['trading_symbol'],
            quantity=quantity,
            validity=groww.VALIDITY_DAY,
            exchange=instrument['exchange'],
            segment=instrument['segment'],
            product=groww.PRODUCT_MIS if intraday else groww.PRODUCT_CNC,
            order_type=groww.ORDER_TYPE_MARKET,
            transaction_type=groww.TRANSACTION_TYPE_SELL,
        )
        print(f"Sell order executed: {orderid}")
        return orderid
    except Exception as e:
        print(e)
        return None
 
def sma_strategy(groww, config):
    """
    Main strategy function implementing Moving Average Crossover trading logic
    
    Strategy Rules:
    1. Wait for entry time
    2. Monitor SMA crossovers
    3. Enter long position on bullish crossover (short period SMA crosses above long period SMA)
    4. Enter short position on bearish crossover (short period SMA crosses below long period SMA)
    5. Exit positions on opposite crossover signals
    6. Square off all positions at exit time
    """
    entry_time = config['entry_time']
    exit_time = config['exit_time']
 
    # Initialize instrument and get stock details
    instruments = Instrument(groww)
    stock = instruments.get_instrument(config['exchange'], config['trading_symbol'])
    
    # Wait for market entry time
    await_entry(entry_time)
 
    # Initialize candle data with 7 days of historical data
    stock_candles = Candles(groww, stock, config['interval'])
    stock_candles.load(duration=7)
    stock_candles.start_auto_update(refresh_interval=5)
    
    position = 0  # 0: no position, 1: long position, -1: short position
 
    # Main trading loop
    while not is_exit_time(exit_time):
        # Get current market direction and crossover status
        direction, cross_over = get_sma(stock_candles, config['sma_short'], config['sma_long'])
        print(direction, cross_over)
        
        # Entry logic: Enter positions only on crossovers when not already positioned
        if position == 0 and cross_over:
            if direction == "BULLISH":
                # Bullish crossover - go long
                buy_at_market(groww, stock, config['quantity'])
                position = 1
            elif direction == "BEARISH":
                # Bearish crossover - go short (sell first, buy later to cover)
                sell_at_market(groww, stock, config['quantity'])
                position = -1
        
        # Exit logic: Exit positions on opposite signals
        elif position == 1 and direction == "BEARISH":
            # Exit long position on bearish signal
            sell_at_market(groww, stock, config['quantity'])
            position = 0
        elif position == -1 and direction == "BULLISH":
            # Cover short position on bullish signal
            buy_at_market(groww, stock, config['quantity'])
            position = 0
        
        time.sleep(5)  # Check signals every 5 seconds
    
    # Square off any remaining positions at exit time
    if position == 1:
        # Exit long position
        sell_at_market(groww, stock, config['quantity'])
    elif position == -1:
        # Cover short position
        buy_at_market(groww, stock, config['quantity'])
 
if __name__ == "__main__":
    # Initialize API and run the strategy
    groww = init_groww(credentials['api_key'], credentials['totp_secret'])
    sma_strategy(groww, config)